<a href="https://colab.research.google.com/github/zahrahimi/ReluplexCav2017/blob/master/bigram_sparse_multiplicative_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk


In [ ]:
# Train and Test Dataset

import nltk
from nltk import word_tokenize, ngrams
nltk.download('punkt')
import re
import numpy as np
import random
import time
import math
from itertools import permutations
# import cyipopt


num_ngram = 3
print("num_ngram = ", num_ngram)

# Specify the file paths
train_file = 'ptb.train.txt'
valid_file = 'ptb.valid.txt'
test_file = 'ptb.test.txt'


# Read the data from the files
def read_data(file_path):
    with open(file_path, 'r') as file:
        data = file.read()
        # Remove special characters and extra spaces
        data = re.sub(r"[^a-zA-Z0-9\s<unk>]", "", data)  ############remove any character that is not a letter, digit, whitespace, <unk>
        # Convert to lowercase
        data = data.lower().strip()
        # Split into sentences and strip leading/trailing spaces from each sentence
        data = [sentence.strip().split() for sentence in data.split('\n')]
        # data = [item for sublist in data for item in  sublist]
        data = [item for sublist in data for item in sublist +['<eos>']]
        # data = [item for sublist in data for item in sublist]

    return data

# Read the training data
train_data_all = read_data(train_file)
print(len(train_data_all))
vocab = sorted(list(set(train_data_all)))
vocab_size = len(vocab)
print('vocab_size:',vocab_size)
print('train_data_all:', train_data_all[:10])
# Read the validation data
valid_data_all = read_data(valid_file)

# Read the test data
test_data_all = read_data(test_file)

#--------------------------------------------trigram

def generate_trigrams(data):
    # Create trigrams by zipping over the data offset by one word each time
    return list(zip(data, data[1:], data[2:]))

test_trigrams = generate_trigrams(test_data_all)
test_size = len(test_trigrams)
print('test_trigrams:', test_trigrams[:10])
print('test_size:', test_size)


train_trigrams = generate_trigrams(train_data_all)
train_size = len(train_trigrams)

#--------------------------------------------bigram
def generate_bigrams(data):
    # Create trigrams by zipping over the data offset by one word each time
    return list(zip(data, data[1:]))


test_bigrams = generate_bigrams(test_data_all)
test_size2 = len(test_bigrams)
print('test_bigrams:', test_bigrams[:10])
print('test_size2:', test_size2)


train_bigrams = generate_bigrams(train_data_all)
train_size2 = len(train_bigrams)


# def split_data(tokens, n):
#     # Tokenize the first document into words
#     # Generate n-grams from the list of tokens
#     # Directly iterate over the generator
#     lst_ngrams = list(ngrams(tokens, n))
#     # Use list comprehension for efficiency
#     lst_xs = [ngram[:-1] for ngram in lst_ngrams]
#     lst_y = [ngram[-1] for ngram in lst_ngrams]

#     return lst_xs, lst_y

#--------------------------------------------my example
# def split_data(tokens, n):
#     # Tokenize the first document into words
#     tokens = word_tokenize(tokens[0])
#     # print('tokens', tokens)

#     # Generate n-grams from the list of tokens
#     # Directly iterate over the generator
#     lst_ngrams = list(ngrams(tokens, n))
#     # Use list comprehension for efficiency
#     lst_xs = [ngram[:-1] for ngram in lst_ngrams]
#     lst_y = [ngram[-1] for ngram in lst_ngrams]

#     return lst_xs, lst_y

# Example usage:
# train_data_all = ["he likes to play tennis"]
# train_data_all = ["he likes to"]
# vocab = sorted(list(set(word_tokenize(train_data_all[0]))))
# print('vocab =', vocab)
# vocab_size = len(vocab)
# print('vocab_size= ', vocab_size)
# print('train_data_all =', train_data_all[0])
#--------------------------------------------

# lst_xs, lst_y = split_data(train_data_all, num_ngram)
# print("lst_xs=", lst_xs)
# print("lst_y=", lst_y)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


num_ngram =  3
920154
vocab_size: 9931
train_data_all: ['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydroquebec']
test_trigrams: [('no', 'it', 'was'), ('it', 'was', 'nt'), ('was', 'nt', 'black'), ('nt', 'black', 'monday'), ('black', 'monday', '<eos>'), ('monday', '<eos>', 'but'), ('<eos>', 'but', 'while'), ('but', 'while', 'the'), ('while', 'the', 'new'), ('the', 'new', 'york')]
test_size: 81673
test_bigrams: [('no', 'it'), ('it', 'was'), ('was', 'nt'), ('nt', 'black'), ('black', 'monday'), ('monday', '<eos>'), ('<eos>', 'but'), ('but', 'while'), ('while', 'the'), ('the', 'new')]
test_size2: 81674


In [ ]:
# Creating Sparse trigram C_train and C_test

from scipy.sparse import coo_array
import numpy as np

def CreateSparseTensor(data, word_to_index, vocab_size, slice_='True'):
    # Lists to accumulate the non-zero values and their coordinates
    values = []  # Non-zero data entries
    row_indices = []  # Row coordinates
    col_indices = []  # Column coordinates (flattened last two dimensions)
    data_size = 0

    # Process the input data to populate 'values' and 'coords'
    for i in range(len(data) - 2):
        word1, word2, word3 = data[i], data[i + 1], data[i + 2]
        if word1 in word_to_index and word2 in word_to_index and word3 in word_to_index:
            i2 = word_to_index[word1]
            i1 = word_to_index[word2]
            j = word_to_index[word3]
            data_size += 1

            # Append the data value (1 for each trigram occurrence)
            values.append(1)

            if slice_ == 'True':
              # Append the row and column indices
              row_indices.append(i2)
              col_indices.append(i1 * vocab_size + j)  # Flatten the 2D indices into 1D

            else:
              row_indices.append(i1)
              col_indices.append(i2 * vocab_size + j)  # Flatten the 2D indices into 1D

    # Convert lists to numpy arrays
    data_np = np.array(values)
    row_np = np.array(row_indices)
    col_np = np.array(col_indices)

    # Create the sparse COO matrix
    tensor_coo = coo_array((data_np, (row_np, col_np)), shape=(vocab_size, vocab_size * vocab_size), dtype=np.float32)
    tensor_csr = tensor_coo.tocsr()         # Removing Duplicates by Converting Formats
    # # Note: Create the sparse CSR matrix, Using vocab_size**2 for the number of columns as the last two dimensions are flattened
    # C_csr = csr_matrix((values, (rows, cols)), shape=(vocab_size, vocab_size**2))

    return tensor_csr


#-------------------------------------
# Example usage:
# vocab = ["0", "1", "2"]
# train_data_all = ["0", "1", "2", "0", "1", "2","1","0","1"]
# test_data_all = ["0", "0", "2", "0"]
# def generate_trigrams(data):
#     # Create trigrams by zipping over the data offset by one word each time
#     return list(zip(data, data[1:], data[2:]))

# test_trigrams = generate_trigrams(test_data_all)
# test_size = len(test_trigrams)
#-------------------------------------


word_to_index = {word: index for index, word in enumerate(vocab)}
vocab_size = len(vocab)

C_train_csr = CreateSparseTensor(train_data_all, word_to_index, vocab_size,'True')
print('C_train_csr_dense:\n', C_train_csr.todense())

C_train_csr_slice2 = CreateSparseTensor(train_data_all, word_to_index, vocab_size,'False')
# print('C_train_csr_dense_slice2:\n', C_train_csr_slice2.todense())

C_test = CreateSparseTensor(test_data_all, word_to_index, vocab_size,'True')
# print('C_test:\n', C_test.todense())

# print('test_size:',test_size)

# print('C_train_csr\n:', C_train_csr)
# print('C_train_csr_dense:\n', C_train_csr.todense())
# print('sparse.nnz:\n', C_train_csr.nnz)
# print('sum on rows\n:', C_train_csr.sum(axis=0))
# print('sum on columns\n:', C_train_csr.sum(axis=1))
# print('data\n:', C_train_csr.data)
# # print('slicing_dense:\n', C_train_csr.todense()[[1], :])
# # print('slicing_sparse:\n', C_train_csr.tocsr()[[1], :])
# # print('transpose_dense:\n', C_train_csr.T.todense())
# # print('transpose_sparse:\n', C_train_csr.T)
# # print('remove zeros:\n', C_train_csr.tocsr().eliminate_zeros())
# print('p1\n:', C_train_csr.sum(axis=0))
# print(type(C_train_csr))

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)
# lambda_matrix = csr_matrix(lambda_matrix)
# print('lambda_matrix:\n',lambda_matrix.todense())


C_train_csr_dense:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Creating Sparse bigram C_train and C_test

from scipy.sparse import coo_array
import numpy as np

def CreateCP(data, word_to_index, vocab_size, slice_):
    # Lists to accumulate the non-zero values and their coordinates
    values = []  # Non-zero data entries
    row_indices = []  # Row coordinates
    col_indices = []  # Column coordinates (flattened last two dimensions)

    # Process the input data to populate 'values' and 'coords'
    for i in range(len(data) - 2):
        word1, word2, word3 = data[i], data[i + 1], data[i + 2]
        if word1 in word_to_index and word2 in word_to_index and word3 in word_to_index:
            i2 = word_to_index[word1]
            i1 = word_to_index[word2]
            j = word_to_index[word3]

            # Append the data value (1 for each trigram occurrence)
            values.append(1)

            if slice_ == 'P1':
              # Append the row and column indices
              row_indices.append(i1)
              col_indices.append(j)

            elif slice_ == 'P2':
              row_indices.append(i2)
              col_indices.append(j)  # Flatten the 2D indices into 1D

    # Convert lists to numpy arrays
    data_np = np.array(values)
    row_np = np.array(row_indices)
    col_np = np.array(col_indices)

    # Create the sparse COO matrix
    tensor_coo = coo_array((data_np, (row_np, col_np)), shape=(vocab_size, vocab_size), dtype=np.float32)
    tensor_csr = tensor_coo.tocsr()         # Removing Duplicates by Converting Formats
    # # Note: Create the sparse CSR matrix, Using vocab_size**2 for the number of columns as the last two dimensions are flattened
    # C_csr = csr_matrix((values, (rows, cols)), shape=(vocab_size, vocab_size**2))

    return tensor_csr

#-------------------------------------
# Example usage:
# vocab = ["0", "1", "2"]
# train_data_all = ["0", "1", "2", "0", "1", "2","1","0","1"]
# test_data_all = ["0", "0", "2", "0", "2"]
# def generate_trigrams(data):
#     # Create trigrams by zipping over the data offset by one word each time
#     return list(zip(data, data[1:], data[2:]))

# test_trigrams = generate_trigrams(test_data_all)
# test_size = len(test_trigrams)


# def generate_bigrams(data):
#     # Create bigrams by zipping over the data offset by one word each time
#     return list(zip(data, data[1:]))

# test_bigrams = generate_bigrams(test_data_all)
# test_size = len(test_bigrams)
#-------------------------------------


word_to_index = {word: index for index, word in enumerate(vocab)}
vocab_size = len(vocab)

CP1 = CreateCP(train_data_all, word_to_index, vocab_size,'P1')
print('CP1:\n', CP1.todense())


CP2 = CreateCP(train_data_all, word_to_index, vocab_size,'P2')
print('CP2:\n', CP2.todense())

# C_train_csr_slice2 = CreateSparseTensor(train_data_all, word_to_index, vocab_size,'False')
# print('C_train_csr_dense_slice2:\n', C_train_csr_slice2.todense())

# CP1_test = CreateCP1(test_data_all, word_to_index, vocab_size,'True')
# print('CP1_test:\n', CP1_test.todense())

# print('test_size:',test_size)

# print('C_train_csr\n:', C_train_csr)
# print('C_train_csr_dense:\n', C_train_csr.todense())
# print('sparse.nnz:\n', C_train_csr.nnz)
# print('sum on rows\n:', C_train_csr.sum(axis=0))
# print('sum on columns\n:', C_train_csr.sum(axis=1))
# print('data\n:', C_train_csr.data)
# # print('slicing_dense:\n', C_train_csr.todense()[[1], :])
# # print('slicing_sparse:\n', C_train_csr.tocsr()[[1], :])
# # print('transpose_dense:\n', C_train_csr.T.todense())
# # print('transpose_sparse:\n', C_train_csr.T)
# # print('remove zeros:\n', C_train_csr.tocsr().eliminate_zeros())
# print('p1\n:', C_train_csr.sum(axis=0))
# print(type(C_train_csr))

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)
# lambda_matrix = csr_matrix(lambda_matrix)
# print('lambda_matrix:\n',lambda_matrix.todense())


CP1:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
CP2:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

def DistinctTokenSparse(CP):
    """
    Computes a distinct token count for each row in a sparse matrix.

    Parameters:
    - CP: A sparse CSR matrix.

    Returns:
    - distinct_tokens: A NumPy array where each element is the sum of elements less than 1
                       and the count of elements greater than 1 for each row in CP.
    """
    # Convert sparse matrix to COO format for element-wise operations
    CP_coo = CP.tocoo()

    # Find elements less than 1 and create a sparse matrix of the same shape with these elements
    elements_less_than_one = csr_matrix((np.where(CP_coo.data < 1, CP_coo.data, 0), (CP_coo.row, CP_coo.col)), shape=CP.shape)

    # Sum these modified values row-wise
    row_sums_less_than_one = np.array(elements_less_than_one.sum(axis=1)).squeeze()

    # For elements greater than 1, we create a binary sparse matrix and sum row-wise
    elements_greater_than_one = csr_matrix((np.where(CP_coo.data >= 1, 1, 0), (CP_coo.row, CP_coo.col)), shape=CP.shape)
    count_greater_than_one_per_row = np.array(elements_greater_than_one.sum(axis=1)).squeeze()

    # Combine the two metrics
    distinct_tokens = row_sums_less_than_one + count_greater_than_one_per_row

    return distinct_tokens.reshape(-1, 1)

# # Example usage
# # P1_normalized_sparse = csr_matrix(np.array([[0.5, 1, 0.3], [1.5, 0.2, 0.0], [0.0, 0.0, 3.0]]))
# print('P1_normalized_sparse:\n', P1_normalized_sparse.todense())
# # Assuming P1_normalized_sparse is your input sparse matrix in CSR format

# P1_normalized_sparse = CP1
# distinct_tokens_sparse = DistinctTokenSparse(P1_normalized_sparse)
# print('Distinct tokens (sparse handling):\n', distinct_tokens_sparse)

# print('P2_normalized_sparse:\n', P2_normalized_sparse.todense())
# distinct_tokens_sparse2 = DistinctTokenSparse(P2_normalized_sparse)
# print('Distinct tokens2 (sparse handling):\n', distinct_tokens_sparse2)

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

def Smoothing_P_sparse(CP_slice, alpha):
    """
    Efficiently process a single slice of CP_slice in sparse CSR format to create
    a corresponding slice of A using sparse operations, ensuring correct division by row sums.

    Parameters:
    - CP_slice: A sparse CSR matrix slice representing part of the data.
    - alpha: Smoothing parameter.

    Returns:
    - A_slice: A sparse CSR matrix after applying the smoothing operation.
    - n_i: Sum of elements in each row, adjusted to avoid division by zero.
    """
    # Ensure CP_slice is in CSR format
    if not isinstance(CP_slice, csr_matrix):
        CP_slice = csr_matrix(CP_slice)

    # Compute row sums (n_i) with a small value added to avoid division by zero
    # n_i = CP_slice.sum(axis=1).A1 + 1e-10
    n_i = CP_slice.sum(axis=1).A1
    n_i[n_i == 0] = 1

    # Compute row sums for normalization
    # row_sums = P1_updated.sum(axis=1)
    # row_sums[row_sums == 0] = 1  # Avoid division by zero
    # # Compute row sums for normalization
    # row_sums = np.sum(P1_reshaped, axis=1, keepdims=True)
    # row_sums[row_sums == 0] = 1  # Avoid division by zero


    # Create masks for elements >= 1 and < 1
    greater_mask = CP_slice.data >= 1
    print('CP_slice:', CP_slice.todense())
    less_mask = CP_slice.data < 1

    # Initialize A_slice as a copy of CP_slice for output
    A_slice = CP_slice.copy()

    # Get the row index for each non-zero element in CP_slice
    row_indices = np.repeat(np.arange(CP_slice.shape[0]), np.diff(CP_slice.indptr))
    # print('CP_slice.indptr', CP_slice.indptr)
    # print('row_indices:/n',row_indices)
    # print('row_indices[greater_mask]:/n',row_indices[greater_mask])
    # print('n_i[row_indices[greater_mask]]:/n',n_i[row_indices[greater_mask]])
    # print('A_slice.data[greater_mask]', A_slice.data[greater_mask])
    # print('greater_mask:', greater_mask, 'CP_slice.indptr:', CP_slice.indptr, 'np.diff(CP_slice.indptr):', np.diff(CP_slice.indptr),'row_indices[greater_mask]', row_indices[greater_mask], n_i[row_indices[greater_mask]])

    # Apply smoothing operation directly on the data of the sparse matrix
    A_slice.data[greater_mask] = (A_slice.data[greater_mask] - alpha) / n_i[row_indices[greater_mask]]
    A_slice.data[less_mask] = (A_slice.data[less_mask] * (1 - alpha)) / n_i[row_indices[less_mask]]

    return A_slice, n_i.reshape(-1, 1)

# Usage example:
# CP_slice should be a sparse CSR matrix and alpha a float
# Example usage assuming P1_normalized_sparse is a sparse CSR matrix
# P1_normalized_sparse = csr_matrix(np.array([[1.5, 2.0, 0.3], [1.5, 0.2, 0.0], [0.0, 0.0, 0.0]]))
# print('P1_normalized_sparse:\n',P1_normalized_sparse)
# print('P1_normalized_dense:\n',P1_normalized_sparse.todense())
# P1_normalized_sparse = CP1_test
# smoothed_P, n_i = Smoothing_P_sparse(P1_normalized_sparse, alpha=0.8)
# print('smoothed_P:\n', smoothed_P.todense())
# print('n_i:\n', n_i)

# print('P2_normalized_dense:\n',P2_normalized_sparse.todense())
# smoothed_P2, n_i2 = Smoothing_P_sparse(P2_normalized_sparse, alpha=0.8)
# print('smoothed_P2:\n', smoothed_P2.todense())
# print('n_i2:\n', n_i2)

# CP_slice: [[0. 3. 0.]
#  [1. 0. 2.]
#  [1. 1. 0.]]
# greater_mask: [ True  True  True  True  True] CP_slice.indptr: [0 1 3 5] np.diff(CP_slice.indptr): [1 2 2] row_indices[greater_mask] [0 1 1 2 2] [3. 3. 3. 2. 2.]
# smoothed_P:
#  [[0.         1.         0.        ]
#  [0.33333334 0.         0.6666667 ]
#  [0.5        0.5        0.        ]]
# n_i:
#  [[3.]
#  [3.]
#  [2.]]

In [ ]:
# Correct one
import numpy as np
import gc
import scipy.sparse as sp
from scipy.sparse import csr_matrix, find


# # Assuming CT, P1, P2, and lambda_matrix are already in CSR format
# CT = C_train_csr_slice2
# print('CT:\n', CT.todense())
# # Set seed for reproducibility
# np.random.seed(42)
# lambda_matrix = np.random.rand(vocab_size, vocab_size)
# lambda_matrix = csr_matrix(lambda_matrix)
# P1 = P1_normalized_sparse
# P2 = P2_normalized_sparse

def get_original_indices(flat_idx, vocab_size):
    """Convert flat index back to (i2, j) index pair"""
    i2 = flat_idx // vocab_size
    j = flat_idx % vocab_size
    return i2, j


def update_p1_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Ensure all matrices are in CSR format for efficient row access and arithmetic operations
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()


    # Initialize P1_updated as a LIL matrix for efficient row-wise operations
    P1_updated = sp.lil_matrix((vocab_size, vocab_size))


    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Directly extract values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the update for P1 based on the formula
        denominator1 = lambda_val * p1_val + (1 - lambda_val) * p2_val
        # denominator[denominator == 0] = 1
        denominator = denominator1 if denominator1 != 0 else 1
        # if denominator != 0:
        updated_value = (c_val * lambda_val * p1_val) / denominator
        P1_updated[i1, j] += updated_value
        # print('i1:', i1, 'i2:',i2, 'j:', j, 'c_val:',c_val, 'p1_val:', p1_val, 'p2_val:', p2_val, 'lambda_val:',lambda_val,'(c_val * lambda_val * p1_val):', (c_val * lambda_val * p1_val), 'denominator:',denominator,'updated_value:',updated_value)

    # # Compute row sums for normalization
    # row_sums = P1_updated.sum(axis=1)
    # row_sums[row_sums == 0] = 1  # Avoid division by zero
    # # print('row', row_sums)
    # # print('1', P1_updated.todense())

    # # Normalize rows
    # P1_updated = P1_updated / row_sums
    # # print('2', P1_updated.todense())

    # Convert P1_updated to CSR for efficient storage and further operations
    P1_updated = P1_updated.tocsr()

    return P1_updated


def update_p2_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Convert sparse matrices to CSR format for efficient access
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()

    # Initialize P2_updated as a LIL matrix for efficient row-wise operations
    P2_updated = sp.lil_matrix((vocab_size, vocab_size))


    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the update for P2 based on the formula
        # denominator = lambda_val * p1_val + (1 - lambda_val) * p2_val + 1e-10
        denominator2 = lambda_val * p1_val + (1 - lambda_val) * p2_val
        denominator = denominator2 if denominator2 != 0 else 1

        # denominator[denominator == 0] = 1
        # if denominator != 0:
        updated_value = (c_val * (1 - lambda_val) * p2_val) / denominator
        P2_updated[i2, j] += updated_value


    # # Compute row sums for normalization
    # row_sums = P2_updated.sum(axis=1)
    # row_sums[row_sums == 0] = 1  # Avoid division by zero
    # # print('row', row_sums)
    # # print('1', P1_updated.todense())

    # # Normalize rows
    # P2_updated = P2_updated / row_sums
    # # print('2', P1_updated.todense())


    # Convert P2_updated to CSR for efficient storage and further operations
    P2_updated = P2_updated.tocsr()

    return P2_updated


def update_lambda_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Convert sparse matrices to CSR format for easier access
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()

    # Initialize P1_updated as a LIL matrix for efficient row-wise operations
    lambda_updated = sp.lil_matrix((vocab_size, vocab_size))

    # Initialize containers to sum contributions to lambda values
    sum_c_vals = sp.lil_matrix((vocab_size, vocab_size))
    sum_lambda_vals = sp.lil_matrix((vocab_size, vocab_size))

    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the denominator for the update formula
        # denominator = lambda_val * p1_val + (1 - lambda_val) * p2_val + 1e-10
        denominator_ = lambda_val * p1_val + (1 - lambda_val) * p2_val
        denominator = denominator_ if denominator_ != 0 else 1

        # if denominator != 0:
            # Calculate the updates to lambda values
        lambda_contribution = (c_val * lambda_val * p1_val) / denominator
        sum_c_vals[i1, i2] += c_val
        sum_lambda_vals[i1, i2] += lambda_contribution

    # Convert LIL matrices back to CSR for final processing
    sum_c_vals = sum_c_vals.tocsr()
    sum_lambda_vals = sum_lambda_vals.tocsr()

    # Direct element-wise operations to update lambda
    non_zero_indices = sum_c_vals.nonzero()
    for i, j in zip(non_zero_indices[0], non_zero_indices[1]):
        # if sum_c_vals[i, j] != 0:
          lambda_updated[i, j] = sum_lambda_vals[i, j] / sum_c_vals[i, j]

    return lambda_updated


# lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('lambda_updated:\n', lambda_updated.todense())

# # Example usage with dummy sparse matrices
# vocab_size = 3
# CT = sp.random(3, 9, density=0.3, format='csr', data_rvs=np.ones)
# P1 = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)
# P2 = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)
# lambda_matrix = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)
###### initial
# CT = C_train_csr_slice2
# print('CT:\n', CT.todense())

# # Example usage
# P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
# # print('P1_normalized_sparse:\n', P1)
# print('P1_normalized_dense:\n', P1.todense())

# P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
# # print('P2_normalized_sparse:\n', P2)
# print('P2_normalized_sparse:\n', P2.todense())

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)

# # Set seed for reproducibility
# # np.random.seed(42)
# # lambda_matrix = np.random.rand(vocab_size, vocab_size)
# lambda_matrix = csr_matrix(lambda_matrix)
# print('lambda_matrix:\n',lambda_matrix.todense())

# P1_updated = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('P1_updated:\n', P1_updated.todense())

# P2_updated = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('P2_updated:\n', P2_updated.todense())


# update_lambda = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('update_lambda:\n', update_lambda.todense())

# Clear memory explicitly
gc.collect()

0

In [ ]:
from scipy.sparse import csr_matrix, find
import math
import gc

# alpha = 0.75
alpha = 0.75
learning_rate = .1  # Learning rate for updates
max_iterations = 5
rep_lambda = 2


###### initial
CT = C_train_csr_slice2
# print('CT:\n', CT.todense())

# Example usage
# P1_initial = SmartInitialization_p_sparse(C_train_csr, vocab_size)

def Smoothed_CP(CP, alpha):
  A_slice, n_i = Smoothing_P_sparse(CP, alpha)
  d_i_sparse = DistinctTokenSparse(CP)
  total_missing_mass = alpha * d_i_sparse/n_i
  # Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
  # We're creating a dense array of the same shape as A_slice1 with each row having the same value
  missing_mass_per_vocab = np.repeat(total_missing_mass / vocab_size, vocab_size, axis=1)
  # Convert the missing_mass_per_vocab to a sparse matrix
  missing_mass_per_vocab_sparse = csr_matrix(missing_mass_per_vocab)
  # Add the missing_mass_per_vocab_sparse to A_slice1 element-wise (smoothing method to add absolute discount to each element and remove zero element)
  P = A_slice + missing_mass_per_vocab_sparse

  return P


P1 = Smoothed_CP(CP1, alpha)
P2 = Smoothed_CP(CP2, alpha)


# # print('P1_normalized_sparse:\n', P1)
# # print('P1_normalized_dense:\n', P1.todense())
# print('P2_normalized_sparse:\n', P2)
# print('P2_normalized_sparse:\n', P2.todense())

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)

# Set seed for reproducibility
np.random.seed(42)

lambda_matrix = np.full((vocab_size, vocab_size), 0.88) #0.88
# lambda_matrix = np.random.rand(vocab_size, vocab_size)
lambda_matrix = csr_matrix(lambda_matrix)
# print('lambda_matrix:\n',lambda_matrix.todense())


# C_test = np.array([
#     [[1, 5, 0],  # Flattened representation of first "row" in 3D space
#     [0, 0, 1],  # Flattened representation of second "row" in 3D space
#     [0, 5, 0]]  # Flattened representation of third "row" in 3D space
#     ,
#     [[2, 3, 0],  # Flattened representation of first "row" in 3D space
#     [0, 5, 0],  # Flattened representation of second "row" in 3D space
#     [7, 1, 0]]  # Flattened representation of third "row" in 3D space
#     ,
#     [[2, 0, 0],  # Flattened representation of first "row" in 3D space
#     [1, 2, 9],  # Flattened representation of second "row" in 3D space
#     [0, 3, 0]]  # Flattened representation of third "row" in 3D space

# , ], dtype = np.float32)


# # Flatten C_test by combining the last two dimensions
# C_test_flattened = C_test.reshape(C_test.shape[0], -1)

# # Example of how the flattened structure looks
# # print("Flattened C_test:")
# # print(C_test_flattened)

# # Create a sparse CSR matrix from the flattened structure
# # First, find the indices of non-zero elements
# rows, cols = np.nonzero(C_test_flattened)
# values = C_test_flattened[rows, cols]

# # Now, create the CSR matrix
# C_test = csr_matrix((values, (rows, cols)), shape=C_test_flattened.shape)
# print('C_test_sparse in dense form:\n', C_test.todense())

# d_i_sparse1 = np.array(
#  [[2.],
#  [1.],
#  [5.]])
# n_i = np.array(
#  [[2.],
#  [10.],
#  [20.]])
# lambda_matrix = np.random.rand(vocab_size, vocab_size)
# # print("initial_lambda_matrix:\n",  lambda_matrix)
# # P1 = SmartInitialization_p1(C_train)
# # P2 = SmartInitialization_p2(C_train)
# P1 = np.random.rand(vocab_size, vocab_size)
# P2 = np.random.rand(vocab_size, vocab_size)

# def MultiplicativeUpdate(C, lambda_matrix, P1, P2, max_iterations, rep_lambda, learning_rate = 1):


def get_empirical_cross_entropy_sparse(C, P1_csr, P2_csr, lambda_csr, N):
    empirical_cross_entropy = 0
    count = 0
    C_nonzero = find(C)
    # Assuming C and P_hat can be aligned or P_hat's necessary values can be efficiently computed:
    for i, idx, c_val in zip(*C_nonzero):
        i2 = i
        i1, j = get_original_indices(idx, vocab_size)
        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0
        # Compute the denominator for the update formula
        p_val = lambda_val * p1_val + (1 - lambda_val) * p2_val
        count += c_val

        if p_val > 0:
            empirical_cross_entropy -= c_val * math.log2(p_val)
        else:
          empirical_cross_entropy -= 0  # Or some default value, or you can raise an error
    return math.pow(2, empirical_cross_entropy / N), count


for iteration in range(max_iterations):

    print('iteration:', iteration)

    # Calculate the denominator: sum of C across the last axis (j)
    CT = C_train_csr_slice2
    # print('CT:\n', CT.todense())

    # # Prevent division by zero by setting zero denominators to 1 (or add a small epsilon)
    # denominator_lambda[denominator_lambda == 0] = 1e-10

    lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
    # Compute the scaled difference
    scaled_diff = (lambda_updated - lambda_matrix).multiply(learning_rate)
    # Compute the new lambda_matrix
    lambda_matrix = lambda_matrix + scaled_diff
    # print('lambda_matrix111:\n',lambda_matrix.todense())

    # # Optional: Convert back to CSR if you had converted it before
    # lambda_matrix = csr_matrix(lambda_matrix_new)


    # multipliers = [3,10]  # You can extend this list to include more multipliers
    # multipliers = list(range(1,10))
    if iteration > rep_lambda:    #only update lambda and P1 and P2 in first iteration and after that iterates only lambda and then after rep_lambda iterates P1 and P2 as well

      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1 or iteration == 15*rep_lambda+1:    #not work
      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1:
      # if iteration > rep_lambda and (iteration - 1) % rep_lambda == 0:   #the condition will be true for iterations like rep_lambda + 1, 2*rep_lambda + 1, 3*rep_lambda + 1, and so on.

      # if iteration == 0:
      #   ## reseting P1 and P2 after rep_lamda so it measns that with smart initialization of P1 and P2 first update lambda with rep_lambda iterations and then reset P1 and P2 since they are good
      #   P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
      #   P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
      #   print('iteration initial',iteration)

      # if any(iteration == m * rep_lambda + 1 for m in multipliers):
      # if iteration == rep_lambda + 1:
      #   P1 = Smoothed_CP(CP1, alpha)
      #   P2 = Smoothed_CP(CP2, alpha)

      #   print('iteration reset',iteration)

      # else:

        print('iteration update',iteration)

        # Update P1
        numerator_P1 = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        P1 = Smoothed_CP(numerator_P1, alpha)

        # Update P2
        numerator_P2 = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        P2 = Smoothed_CP(numerator_P2, alpha)

        # if iteration % 2 == 0:
        #   P_hat = find_P_hat(lambda_matrix, P1, P2)
        #   # print('P_hat:', P_hat)
        #   N = test_size
        #   empirical_cross_entropy = get_empirical_cross_entropy_sparse(C_test, P_hat, N)
        #   print('empirical_cross_entropy:', empirical_cross_entropy)
            # Clear memory explicitly
        gc.collect()


# print('lambda_matrix_updated:\n',lambda_matrix)

# print('P1_updated:\n', P1)
# print('P1_updated_dense:\n', P1.todense())

# print('P2_updated:\n', P2)
# print('P2_updated_dense:\n', P2.todense())

# def find_P_hat(lambda_matrix, P1, P2):
# # Initialize the 3-dimensional tensor P_hat
#   P_hat = np.zeros((vocab_size, vocab_size, vocab_size))  # Tensor for i2, i1, j

#   # Populate the tensor P using the provided equation
#   for i2 in range(vocab_size):
#       for i1 in range(vocab_size):
#           for j in range(vocab_size):
#               P_hat[i2, i1, j] = lambda_matrix[i1, i2] * P1[i1, j] + (1 - lambda_matrix[i1, i2]) * P2[i2, j]

#   # print("Tensor P_hat:\n", P_hat)
#   return P_hat

# P_hat = find_P_hat(lambda_matrix, P1, P2)



# # Assuming lambda_matrix, P1, P2 are defined sparse matrices
# P_hat = find_P_hat(lambda_matrix, P1, P2)

# print('-----------------------------------------')

N = test_size
# N = train_size
empirical_cross_entropy, count = get_empirical_cross_entropy_sparse(C_test, P1, P2, lambda_matrix, N)
print('empirical_cross_entropy:', empirical_cross_entropy)
# print('count:', count)

# A_slice1, n_i = Smoothing_P_sparse(numerator_P1, alpha=0.8)
# empirical_cross_entropy: 276.6979117176241
# emp2 892.2182737491097
# count: 67598
# N: 81673
# test_size: 81673

# empirical_cross_entropy: 229.33223582100675


CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration update 3
CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
iteration: 4
iteration update 4
CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0.

In [ ]:
# Creating Sparse bigram C_train and C_test.  I get 252 perplexity with this

from scipy.sparse import coo_array
import numpy as np

def CreateCP1(data, word_to_index, vocab_size, slice_='True'):
    # Lists to accumulate the non-zero values and their coordinates
    values = []  # Non-zero data entries
    row_indices = []  # Row coordinates
    col_indices = []  # Column coordinates (flattened last two dimensions)

    # Process the input data to populate 'values' and 'coords'
    for i in range(len(data) - 1):
        word1, word2 = data[i], data[i + 1]
        if word1 in word_to_index and word2 in word_to_index:
            i2 = word_to_index[word1]
            i1 = word_to_index[word2]

            # Append the data value (1 for each trigram occurrence)
            values.append(1)

            if slice_ == 'True':
              # Append the row and column indices
              row_indices.append(i2)
              col_indices.append(i1)
            # else:
            #   row_indices.append(i1)
            #   col_indices.append(i2 * vocab_size + j)  # Flatten the 2D indices into 1D

    # Convert lists to numpy arrays
    data_np = np.array(values)
    row_np = np.array(row_indices)
    col_np = np.array(col_indices)

    # Create the sparse COO matrix
    tensor_coo = coo_array((data_np, (row_np, col_np)), shape=(vocab_size, vocab_size), dtype=np.float32)
    tensor_csr = tensor_coo.tocsr()         # Removing Duplicates by Converting Formats
    # # Note: Create the sparse CSR matrix, Using vocab_size**2 for the number of columns as the last two dimensions are flattened
    # C_csr = csr_matrix((values, (rows, cols)), shape=(vocab_size, vocab_size**2))

    return tensor_csr


def CreateCP2(data, word_to_index, vocab_size, slice_='True'):
    # Lists to accumulate the non-zero values and their coordinates
    values = []  # Non-zero data entries
    row_indices = []  # Row coordinates
    col_indices = []  # Column coordinates (flattened last two dimensions)

    # Process the input data to populate 'values' and 'coords'
    for i in range(len(data) - 2):
        word1, word2, word3 = data[i], data[i + 1], data[i + 2]
        if word1 in word_to_index and word2 in word_to_index and word3 in word_to_index:
            i2 = word_to_index[word1]
            i1 = word_to_index[word2]
            j = word_to_index[word3]

            # Append the data value (1 for each trigram occurrence)
            values.append(1)

            if slice_ == 'True':
              # Append the row and column indices
              row_indices.append(i2)
              col_indices.append(j)
            # else:
            #   row_indices.append(i1)
            #   col_indices.append(i2 * vocab_size + j)  # Flatten the 2D indices into 1D

    # Convert lists to numpy arrays
    data_np = np.array(values)
    row_np = np.array(row_indices)
    col_np = np.array(col_indices)

    # Create the sparse COO matrix
    tensor_coo = coo_array((data_np, (row_np, col_np)), shape=(vocab_size, vocab_size), dtype=np.float32)
    tensor_csr = tensor_coo.tocsr()         # Removing Duplicates by Converting Formats
    # # Note: Create the sparse CSR matrix, Using vocab_size**2 for the number of columns as the last two dimensions are flattened
    # C_csr = csr_matrix((values, (rows, cols)), shape=(vocab_size, vocab_size**2))

    return tensor_csr



#-------------------------------------
# Example usage:
# vocab = ["0", "1", "2"]
# train_data_all = ["0", "1", "2", "0", "1", "2","1","0","1"]
# test_data_all = ["0", "0", "2", "0", "2"]
# def generate_trigrams(data):
#     # Create trigrams by zipping over the data offset by one word each time
#     return list(zip(data, data[1:], data[2:]))

# test_trigrams = generate_trigrams(test_data_all)
# test_size = len(test_trigrams)


# def generate_bigrams(data):
#     # Create bigrams by zipping over the data offset by one word each time
#     return list(zip(data, data[1:]))

# test_bigrams = generate_bigrams(test_data_all)
# test_size = len(test_bigrams)
#-------------------------------------


word_to_index = {word: index for index, word in enumerate(vocab)}
vocab_size = len(vocab)

CP1 = CreateCP1(train_data_all, word_to_index, vocab_size,'True')
print('CP1:\n', CP1.todense())


CP2 = CreateCP2(train_data_all, word_to_index, vocab_size,'True')
print('CP2:\n', CP2.todense())

# C_train_csr_slice2 = CreateSparseTensor(train_data_all, word_to_index, vocab_size,'False')
# print('C_train_csr_dense_slice2:\n', C_train_csr_slice2.todense())

# CP1_test = CreateCP1(test_data_all, word_to_index, vocab_size,'True')
# print('CP1_test:\n', CP1_test.todense())

# print('test_size:',test_size)

# print('C_train_csr\n:', C_train_csr)
# print('C_train_csr_dense:\n', C_train_csr.todense())
# print('sparse.nnz:\n', C_train_csr.nnz)
# print('sum on rows\n:', C_train_csr.sum(axis=0))
# print('sum on columns\n:', C_train_csr.sum(axis=1))
# print('data\n:', C_train_csr.data)
# # print('slicing_dense:\n', C_train_csr.todense()[[1], :])
# # print('slicing_sparse:\n', C_train_csr.tocsr()[[1], :])
# # print('transpose_dense:\n', C_train_csr.T.todense())
# # print('transpose_sparse:\n', C_train_csr.T)
# # print('remove zeros:\n', C_train_csr.tocsr().eliminate_zeros())
# print('p1\n:', C_train_csr.sum(axis=0))
# print(type(C_train_csr))

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)
# lambda_matrix = csr_matrix(lambda_matrix)
# print('lambda_matrix:\n',lambda_matrix.todense())


CP1:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
CP2:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
lambda_matrix.max()


NameError: name 'lambda_matrix' is not defined

In [ ]:
lambda_matrix.min()

0.792

In [ ]:
# Step 1: Compute the mean
matrix = lambda_matrix
mean = np.mean(matrix)

# Step 2: Compute the squared differences
squared_diff = (matrix - mean) ** 2

# Step 3: Compute the variance
variance = np.mean(squared_diff)

# Step 4: Compute the standard deviation
std_deviation = np.sqrt(variance)

print("Standard deviation of the matrix:", std_deviation)

In [ ]:
from scipy.sparse import csr_matrix, find
import math
import gc

alpha = 0.75
learning_rate = .1  # Learning rate for updates
max_iterations = 0
rep_lambda = 5


P1_initial = CP1
A_slice1, n_i = Smoothing_P_sparse(P1_initial, alpha=0.8)
# print('n_i:', type(n_i))
# if np.any(n_i != 1):
#     print("There is at least one element in the matrix that is not equal to 1.")
# else:
#     print("All elements in the matrix are equal to 1.")
d_i_sparse1 = DistinctTokenSparse(P1_initial)
total_missing_mass1 = alpha * d_i_sparse1/n_i

# Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
# We're creating a dense array of the same shape as A_slice1 with each row having the same value
missing_mass_per_vocab = np.repeat(total_missing_mass1 / vocab_size, vocab_size, axis=1)
# Convert the missing_mass_per_vocab to a sparse matrix
missing_mass_per_vocab_sparse = csr_matrix(missing_mass_per_vocab)

# Add the missing_mass_per_vocab_sparse to A_slice1 element-wise (smoothing method to add absolute discount to each element and remove zero element)
P1 = A_slice1 + missing_mass_per_vocab_sparse


def get_empirical_cross_entropy_sparse(C, P1_csr, N):
    empirical_cross_entropy = 0
    count = 0
    C_nonzero = find(C)
    # Assuming C and P_hat can be aligned or P_hat's necessary values can be efficiently computed:
    for i, idx, c_val in zip(*C_nonzero):
        i2 = i
        i1 = idx
        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i2, i1]
        # p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        # lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0
        # Compute the denominator for the update formula
        # p_val = lambda_val * p1_val + (1 - lambda_val) * p2_val
        p_val = p1_val
        count += c_val

        if p_val > 0:
            empirical_cross_entropy -= c_val * math.log2(p_val)
        else:
          empirical_cross_entropy -= 0  # Or some default value, or you can raise an error
    return math.pow(2, empirical_cross_entropy / N), count


N = test_size2
# N = train_size
empirical_cross_entropy, count = get_empirical_cross_entropy_sparse(CP1, P1, N)
print('empirical_cross_entropy:', empirical_cross_entropy)

CP_slice: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
empirical_cross_entropy: 3.607668151910993e+22


In [ ]:
# final code for sparse p1 and p2

from scipy.sparse import csr_matrix
import numpy as np

# print('lambda_matrix:\n',lambda_matrix.todense())
# print('C_train_csr_dense:\n', C_train_csr.todense())

def SmartInitialization_p_sparse(C_train_csr, vocab_size):
    """
    Normalizes a sparse CSR matrix by summing along axis=0 and then normalizing along a specific dimension.

    :param C_train_csr: Sparse matrix in CSR format representing summed data.
    :param vocab_size: The size of the vocabulary, used for understanding the structure.
    :return: A sparse CSR matrix where each row is normalized by its sum.
    """
    # Sum along columns to collapse the matrix into a single row (vocab_size, )
    P1_sum = C_train_csr.sum(axis=0)
    # print('P1_sum =', P1_sum)

    # For a CSR matrix, P1_sum here is a 1xN matrix. We need to convert it to a dense format for further manipulation.
    # Note: This conversion to dense is necessary for division and reshaping operations that follow.
    P1_dense = np.array(P1_sum).squeeze()  # Convert to dense and flatten
    # print('P1_dense =', P1_dense)

    # Reshape P1_dense back into the (vocab_size, vocab_size) structure
    # This step assumes that the vocab_size**2 matches the total number of columns in C_train_csr
    P1_reshaped = P1_dense.reshape(vocab_size, vocab_size)
    # print('P1_reshaped =', P1_reshaped)

    # Compute row sums for normalization
    row_sums = np.sum(P1_reshaped, axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # Avoid division by zero

    # Normalize rows
    P1_normalized = P1_reshaped / row_sums

    # If needed, convert P1_normalized back to a sparse matrix
    # Note: Depending on subsequent use, you might keep it dense or convert back to sparse
    P1_sparse_normalized = csr_matrix(P1_normalized)
    # P1_sparse_normalized = P1_normalized

    return P1_sparse_normalized

# Example usage
P1_normalized_sparse = SmartInitialization_p_sparse(C_train_csr, vocab_size)
# print('P1_normalized_sparse:\n', P1_normalized_sparse)
print('P1_normalized_dense:\n', P1_normalized_sparse.todense())

P2_normalized_sparse = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
print('P2_normalized_sparse:\n', P2_normalized_sparse.todense())
# print('P2_normalized_sparse:\n', P2_normalized_sparse)

# C_train_csr_dense:
#  [[0. 0. 0. 0. 0. 2. 0. 0. 0.]
#  [0. 1. 0. 0. 0. 0. 1. 1. 0.]
#  [0. 1. 0. 1. 0. 0. 0. 0. 0.]]

P1_normalized_dense:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
P2_normalized_sparse:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


the main code

In [ ]:
# Correct one
import numpy as np
import gc
import scipy.sparse as sp

# # Assuming CT, P1, P2, and lambda_matrix are already in CSR format
# CT = C_train_csr_slice2
# print('CT:\n', CT.todense())
# # Set seed for reproducibility
# np.random.seed(42)
# lambda_matrix = np.random.rand(vocab_size, vocab_size)
# lambda_matrix = csr_matrix(lambda_matrix)
# P1 = P1_normalized_sparse
# P2 = P2_normalized_sparse

def get_original_indices(flat_idx, vocab_size):
    """Convert flat index back to (i2, j) index pair"""
    i2 = flat_idx // vocab_size
    j = flat_idx % vocab_size
    return i2, j


def update_p1_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Ensure all matrices are in CSR format for efficient row access and arithmetic operations
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()


    # Initialize P1_updated as a LIL matrix for efficient row-wise operations
    P1_updated = sp.lil_matrix((vocab_size, vocab_size))

    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Directly extract values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the update for P1 based on the formula
        denominator = lambda_val * p1_val + (1 - lambda_val) * p2_val + 1e-10
        # if denominator != 0:
        updated_value = (c_val * lambda_val * p1_val) / denominator
        P1_updated[i1, j] += updated_value

    # Convert P1_updated to CSR for efficient storage and further operations
    P1_updated = P1_updated.tocsr()

    return P1_updated


def update_p2_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Convert sparse matrices to CSR format for efficient access
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()

    # Initialize P2_updated as a LIL matrix for efficient row-wise operations
    P2_updated = sp.lil_matrix((vocab_size, vocab_size))

    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the update for P2 based on the formula
        denominator = lambda_val * p1_val + (1 - lambda_val) * p2_val + 1e-10
        # if denominator != 0:
        updated_value = (c_val * (1 - lambda_val) * p2_val) / denominator
        P2_updated[i2, j] += updated_value

    # Convert P2_updated to CSR for efficient storage and further operations
    P2_updated = P2_updated.tocsr()

    return P2_updated


def update_lambda_sparse(CT, P1_csr, P2_csr, lambda_csr, vocab_size):
    # Convert sparse matrices to CSR format for easier access
    # CT_csr = CT.tocsr()
    # P1_csr = P1.tocsr()
    # P2_csr = P2.tocsr()
    # lambda_csr = lambda_matrix.tocsr()

    # Initialize P1_updated as a LIL matrix for efficient row-wise operations
    lambda_updated = sp.lil_matrix((vocab_size, vocab_size))

    # Initialize containers to sum contributions to lambda values
    sum_c_vals = sp.lil_matrix((vocab_size, vocab_size))
    sum_lambda_vals = sp.lil_matrix((vocab_size, vocab_size))

    # Iterate through non-zero elements in CT
    CT_nonzero = find(CT)
    for i, idx, c_val in zip(*CT_nonzero):
        i1 = i
        i2, j = get_original_indices(idx, vocab_size)

        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0

        # Compute the denominator for the update formula
        denominator = lambda_val * p1_val + (1 - lambda_val) * p2_val + 1e-10
        # if denominator != 0:
            # Calculate the updates to lambda values
        lambda_contribution = (c_val * lambda_val * p1_val) / denominator
        sum_c_vals[i1, i2] += c_val
        sum_lambda_vals[i1, i2] += lambda_contribution

    # Convert LIL matrices back to CSR for final processing
    sum_c_vals = sum_c_vals.tocsr()
    sum_lambda_vals = sum_lambda_vals.tocsr()

    # Direct element-wise operations to update lambda
    non_zero_indices = sum_c_vals.nonzero()
    for i, j in zip(non_zero_indices[0], non_zero_indices[1]):
        # if sum_c_vals[i, j] != 0:
          lambda_updated[i, j] = sum_lambda_vals[i, j] / sum_c_vals[i, j]

    return lambda_updated


# lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('lambda_updated:\n', lambda_updated.todense())

# # Example usage with dummy sparse matrices
# vocab_size = 3
# CT = sp.random(3, 9, density=0.3, format='csr', data_rvs=np.ones)
# P1 = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)
# P2 = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)
# lambda_matrix = sp.random(3, 3, density=0.3, format='csr', data_rvs=np.ones)

# P1_updated = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('P1_updated:\n', P1_updated.todense())

# P2_updated = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# print('P2_updated:\n', P2_updated.todense())
# Clear memory explicitly
gc.collect()

0

In [ ]:
P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
print('P1_normalized_dense:\n',P1_normalized_sparse.todense())
P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
print('P2_normalized_dense:\n',P2_normalized_sparse.todense())

lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
# Compute the scaled difference
scaled_diff = (lambda_updated - lambda_matrix).multiply(learning_rate)
# Compute the new lambda_matrix
lambda_matrix = lambda_matrix + scaled_diff

KeyboardInterrupt: 

In [ ]:
# Update P1
alpha = 0.8
learning_rate = .1  # Learning rate for updates
CT = C_train_csr_slice2

P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
print('P1_normalized_dense:\n',P1_normalized_sparse.todense())
P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
# print('P2_normalized_dense:\n',P2_normalized_sparse.todense())

numerator_P1 = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
print('numerator_P1:\n',numerator_P1.todense())
A_slice1, n_i = Smoothing_P_sparse(numerator_P1, alpha=0.8)
print('A_slice1:\n',A_slice1.todense(), '\nn_i:\n', n_i)
d_i_sparse1 = DistinctTokenSparse(numerator_P1)
print('d_i_sparse1:\n',d_i_sparse1)
total_missing_mass1 = alpha * d_i_sparse1/n_i
print('total_missing_mass1:\n',total_missing_mass1)

# Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
# We're creating a dense array of the same shape as A_slice1 with each row having the same value
missing_mass_per_vocab = np.repeat(total_missing_mass1 / vocab_size, vocab_size, axis=1)
# Convert the missing_mass_per_vocab to a sparse matrix
missing_mass_per_vocab_sparse = csr_matrix(missing_mass_per_vocab)

# Add the missing_mass_per_vocab_sparse to A_slice1 element-wise (smoothing method to add absolute discount to each element and remove zero element)
P1 = A_slice1 + missing_mass_per_vocab_sparse
print('P11111:\n',P1.todense())

P1_normalized_dense:
 [[0.         1.         0.        ]
 [0.33333334 0.         0.6666667 ]
 [0.5        0.5        0.        ]]
numerator_P1:
 [[0.00000000e+00 7.63021894e-01 0.00000000e+00]
 [6.53284855e-04 0.00000000e+00 4.35838714e-02]
 [4.04962378e-01 2.53888919e-01 0.00000000e+00]]
A_slice1:
 [[0.         0.2        0.        ]
 [0.00295356 0.         0.19704644]
 [0.12292983 0.07707017 0.        ]] 
n_i:
 [[0.76302189]
 [0.04423716]
 [0.6588513 ]]
d_i_sparse1:
 [[0.76302189]
 [0.04423716]
 [0.6588513 ]]
total_missing_mass1:
 [[0.8]
 [0.8]
 [0.8]]
P11111:
 [[0.26666667 0.46666667 0.26666667]
 [0.26962022 0.26666667 0.46371311]
 [0.38959649 0.34373684 0.26666667]]


In [ ]:
# # Update P2

P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
# print('P1_normalized_dense:\n',P1_normalized_sparse.todense())
P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
print('P2_normalized_dense:\n',P2_normalized_sparse.todense())
CT = C_train_csr_slice2
print('CT:\n',CT.todense())

numerator_P2 = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
print('numerator_P2:\n',numerator_P2.todense())
A_slice2, n_i = Smoothing_P_sparse(numerator_P2, alpha=0.8)
print('A_slice2:\n',A_slice2.todense(), '\nn_i:\n', n_i)
d_i_sparse2 = DistinctTokenSparse(numerator_P2)
print('d_i_sparse2:\n',d_i_sparse2)
total_missing_mass2 = alpha * d_i_sparse2/n_i
print('total_missing_mass2:\n',total_missing_mass2)
# Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
# We're creating a dense array of the same shape as A_slice1 with each row having the same value
missing_mass_per_vocab2 = np.repeat(total_missing_mass2 / vocab_size, vocab_size, axis=1)
# Convert the missing_mass_per_vocab to a sparse matrix
missing_mass_per_vocab_sparse2 = csr_matrix(missing_mass_per_vocab2)
P2 = A_slice2 + missing_mass_per_vocab_sparse2
print('P22222:\n',P2.todense())

P2_normalized_dense:
 [[0.         0.         1.        ]
 [0.33333334 0.6666667  0.        ]
 [0.5        0.5        0.        ]]
CT:
 [[0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 2. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0.]]
numerator_P2:
 [[0.         0.         1.95641613]
 [0.59503762 1.3566861  0.        ]
 [0.99934671 0.62640309 0.        ]]
A_slice2:
 [[0.         0.         0.59108904]
 [0.0609756  0.28522792 0.        ]
 [0.12293979 0.07706021 0.        ]] 
n_i:
 [[1.95641613]
 [1.95172372]
 [1.6257498 ]]
d_i_sparse2:
 [[1.        ]
 [1.59503762]
 [1.6257498 ]]
total_missing_mass2:
 [[0.40891096]
 [0.65379648]
 [0.8       ]]
P22222:
 [[0.13630365 0.13630365 0.72739269]
 [0.27890776 0.50316008 0.21793216]
 [0.38960646 0.34372687 0.26666667]]


In [ ]:
from scipy.sparse import csr_matrix, find
import math
import gc

alpha = 0.8
learning_rate = .1  # Learning rate for updates
max_iterations = 11
rep_lambda = 1


###### initial
CT = C_train_csr_slice2
# print('CT:\n', CT.todense())

# Example usage
P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
# print('P1_normalized_sparse:\n', P1)
# print('P1_normalized_dense:\n', P1.todense())

P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
# print('P2_normalized_sparse:\n', P2)
# print('P2_normalized_sparse:\n', P2.todense())

# lambda_matrix = np.array(
#     [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
#     [0, 5, 4],  # Flattened representation of second "row" in 3D space
#     [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
#  , dtype = np.float32)

# Set seed for reproducibility
np.random.seed(42)
lambda_matrix = np.random.rand(vocab_size, vocab_size)
lambda_matrix = csr_matrix(lambda_matrix)
print('lambda_matrix:\n',lambda_matrix.todense())


# C_test = np.array([
#     [[1, 5, 0],  # Flattened representation of first "row" in 3D space
#     [0, 0, 1],  # Flattened representation of second "row" in 3D space
#     [0, 5, 0]]  # Flattened representation of third "row" in 3D space
#     ,
#     [[2, 3, 0],  # Flattened representation of first "row" in 3D space
#     [0, 5, 0],  # Flattened representation of second "row" in 3D space
#     [7, 1, 0]]  # Flattened representation of third "row" in 3D space
#     ,
#     [[2, 0, 0],  # Flattened representation of first "row" in 3D space
#     [1, 2, 9],  # Flattened representation of second "row" in 3D space
#     [0, 3, 0]]  # Flattened representation of third "row" in 3D space

# , ], dtype = np.float32)


# # Flatten C_test by combining the last two dimensions
# C_test_flattened = C_test.reshape(C_test.shape[0], -1)

# # Example of how the flattened structure looks
# # print("Flattened C_test:")
# # print(C_test_flattened)

# # Create a sparse CSR matrix from the flattened structure
# # First, find the indices of non-zero elements
# rows, cols = np.nonzero(C_test_flattened)
# values = C_test_flattened[rows, cols]

# # Now, create the CSR matrix
# C_test = csr_matrix((values, (rows, cols)), shape=C_test_flattened.shape)
# print('C_test_sparse in dense form:\n', C_test.todense())

# d_i_sparse1 = np.array(
#  [[2.],
#  [1.],
#  [5.]])
# n_i = np.array(
#  [[2.],
#  [10.],
#  [20.]])
# lambda_matrix = np.random.rand(vocab_size, vocab_size)
# # print("initial_lambda_matrix:\n",  lambda_matrix)
# # P1 = SmartInitialization_p1(C_train)
# # P2 = SmartInitialization_p2(C_train)
# P1 = np.random.rand(vocab_size, vocab_size)
# P2 = np.random.rand(vocab_size, vocab_size)

# def MultiplicativeUpdate(C, lambda_matrix, P1, P2, max_iterations, rep_lambda, learning_rate = 1):


def get_empirical_cross_entropy_sparse(C, P1_csr, P2_csr, lambda_csr, N):
    empirical_cross_entropy = 0
    count = 0
    C_nonzero = find(C)
    # Assuming C and P_hat can be aligned or P_hat's necessary values can be efficiently computed:
    for i, idx, c_val in zip(*C_nonzero):
        i2 = i
        i1, j = get_original_indices(idx, vocab_size)
        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0
        # Compute the denominator for the update formula
        p_val = lambda_val * p1_val + (1 - lambda_val) * p2_val
        count += c_val

        if p_val > 0:
            empirical_cross_entropy -= c_val * math.log2(p_val)
        else:
          empirical_cross_entropy -= 0  # Or some default value, or you can raise an error
    return math.pow(2, empirical_cross_entropy / N), count


for iteration in range(max_iterations):

    print('iteration:', iteration)

    # Calculate the denominator: sum of C across the last axis (j)
    CT = C_train_csr_slice2
    # print('CT:\n', CT.todense())

    # # Prevent division by zero by setting zero denominators to 1 (or add a small epsilon)
    # denominator_lambda[denominator_lambda == 0] = 1e-10

    lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
    # Compute the scaled difference
    scaled_diff = (lambda_updated - lambda_matrix).multiply(learning_rate)
    # Compute the new lambda_matrix
    lambda_matrix = lambda_matrix + scaled_diff
    # print('lambda_matrix111:\n',lambda_matrix.todense())

    # # Optional: Convert back to CSR if you had converted it before
    # lambda_matrix = csr_matrix(lambda_matrix_new)


    multipliers = [5,10]  # You can extend this list to include more multipliers
    # multipliers = list(range(1,10))
    if iteration == 0 or iteration > rep_lambda:    #only update lambda and P1 and P2 in first iteration and after that iterates only lambda and then after rep_lambda iterates P1 and P2 as well

      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1 or iteration == 15*rep_lambda+1:    #not work
      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1:
      # if iteration > rep_lambda and (iteration - 1) % rep_lambda == 0:   #the condition will be true for iterations like rep_lambda + 1, 2*rep_lambda + 1, 3*rep_lambda + 1, and so on.

      if iteration == 0:
        ## reseting P1 and P2 after rep_lamda so it measns that with smart initialization of P1 and P2 first update lambda with rep_lambda iterations and then reset P1 and P2 since they are good
        P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
        P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
        print('iteration initial',iteration)

      if any(iteration == m * rep_lambda + 1 for m in multipliers):
        P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
        P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)

        print('iteration reset',iteration)

      elif iteration != 0:

        print('iteration update',iteration)

        # Update P1
        numerator_P1 = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        A_slice1, n_i = Smoothing_P_sparse(numerator_P1, alpha=0.8)
        d_i_sparse1 = DistinctTokenSparse(numerator_P1)
        total_missing_mass1 = alpha * d_i_sparse1/n_i

        # Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
        # We're creating a dense array of the same shape as A_slice1 with each row having the same value
        missing_mass_per_vocab = np.repeat(total_missing_mass1 / vocab_size, vocab_size, axis=1)
        # Convert the missing_mass_per_vocab to a sparse matrix
        missing_mass_per_vocab_sparse = csr_matrix(missing_mass_per_vocab)

        # Add the missing_mass_per_vocab_sparse to A_slice1 element-wise (smoothing method to add absolute discount to each element and remove zero element)
        P1 = A_slice1 + missing_mass_per_vocab_sparse
        print('P11111:\n',P1.todense())


        # # Update P2
        numerator_P2 = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        A_slice2, n_i = Smoothing_P_sparse(numerator_P2, alpha=0.8)
        d_i_sparse2 = DistinctTokenSparse(numerator_P2)
        total_missing_mass2 = alpha * d_i_sparse2/n_i
        # Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
        # We're creating a dense array of the same shape as A_slice1 with each row having the same value
        missing_mass_per_vocab2 = np.repeat(total_missing_mass2 / vocab_size, vocab_size, axis=1)
        # Convert the missing_mass_per_vocab to a sparse matrix
        missing_mass_per_vocab_sparse2 = csr_matrix(missing_mass_per_vocab2)
        P2 = A_slice2 + missing_mass_per_vocab_sparse2
        print('P22222:\n',P2.todense())
        # P2 = numerator_P2 / np.sum(numerator_P2, axis=1, keepdims=True)
        # P2 /= P2.sum(axis=1, keepdims=True)
    # if iteration % 2 == 0:
    #   P_hat = find_P_hat(lambda_matrix, P1, P2)
    #   # print('P_hat:', P_hat)
    #   N = test_size
    #   empirical_cross_entropy = get_empirical_cross_entropy_sparse(C_test, P_hat, N)
    #   print('empirical_cross_entropy:', empirical_cross_entropy)
        # Clear memory explicitly
        gc.collect()


# print('lambda_matrix_updated:\n',lambda_matrix)

# print('P1_updated:\n', P1)
# print('P1_updated_dense:\n', P1.todense())

# print('P2_updated:\n', P2)
# print('P2_updated_dense:\n', P2.todense())

# def find_P_hat(lambda_matrix, P1, P2):
# # Initialize the 3-dimensional tensor P_hat
#   P_hat = np.zeros((vocab_size, vocab_size, vocab_size))  # Tensor for i2, i1, j

#   # Populate the tensor P using the provided equation
#   for i2 in range(vocab_size):
#       for i1 in range(vocab_size):
#           for j in range(vocab_size):
#               P_hat[i2, i1, j] = lambda_matrix[i1, i2] * P1[i1, j] + (1 - lambda_matrix[i1, i2]) * P2[i2, j]

#   # print("Tensor P_hat:\n", P_hat)
#   return P_hat

# P_hat = find_P_hat(lambda_matrix, P1, P2)



# # Assuming lambda_matrix, P1, P2 are defined sparse matrices
# P_hat = find_P_hat(lambda_matrix, P1, P2)

# print('-----------------------------------------')

N = test_size
empirical_cross_entropy, count = get_empirical_cross_entropy_sparse(C_test, P1, P2, lambda_matrix, N)
print('empirical_cross_entropy:', empirical_cross_entropy)
# print('count:', count)


# empirical_cross_entropy: 276.6979117176241
# emp2 892.2182737491097
# count: 67598
# N: 81673
# test_size: 81673

lambda_matrix:
 [[0.37454012 0.95071431 0.73199394 ... 0.83522929 0.89345905 0.82322636]
 [0.10304169 0.22744825 0.00312165 ... 0.71613506 0.73364407 0.93356231]
 [0.20907129 0.99456557 0.7734812  ... 0.90922276 0.30271467 0.75398341]
 ...
 [0.5504548  0.48085988 0.69397494 ... 0.26938843 0.95902593 0.66106117]
 [0.59110639 0.4921623  0.23967714 ... 0.76942664 0.8686995  0.77491025]
 [0.7822393  0.62603131 0.55499552 ... 0.25877062 0.79041964 0.40993842]]
iteration: 0
iteration initial 0
iteration: 1
iteration: 2
iteration update 2
P11111:
 [[4.61194501e-06 4.61194501e-06 4.61194501e-06 ... 4.61194501e-06
  4.61194501e-06 4.61194501e-06]
 [2.93494020e-05 2.93494020e-05 2.93494020e-05 ... 2.93494020e-05
  2.93494020e-05 2.93494020e-05]
 [5.36513647e-05 5.36513647e-05 5.36513647e-05 ... 5.36513647e-05
  5.36513647e-05 5.36513647e-05]
 ...
 [8.05558353e-05 8.05558353e-05 8.05558353e-05 ... 8.05558353e-05
  8.05558353e-05 8.05558353e-05]
 [8.05558353e-05 8.05558353e-05 8.05558353e-05 ... 8

In [ ]:
# Final version how to create P with for loop
import numpy as np
import random

lambda_matrix_target = np.array(
    [[1, 0, 0.3],  # Flattened representation of first "row" in 3D space
    [0, 5, 4],  # Flattened representation of second "row" in 3D space
    [0.1, 3, 0]]  # Flattened representation of third "row" in 3D space
 , dtype = np.float32)
print('lambda_matrix_target:\n',lambda_matrix_target)

# Check that the rows sum to 1
P1_target = np.array(
    [[0.7, 0, 0.3],  # Flattened representation of first "row" in 3D space
    [0.1, .5, .4],  # Flattened representation of second "row" in 3D space
    [1, 0, 0]]  # Flattened representation of third "row" in 3D space
 , dtype = np.float32)
print("target_P1:\n", P1_target)

P2_target = np.array(
    [[.1, 0.6, 0.3],  # Flattened representation of first "row" in 3D space
    [0, 1, 0],  # Flattened representation of second "row" in 3D space
    [0.1, 0.5, 0.4]]  # Flattened representation of third "row" in 3D space
 , dtype = np.float32)
print("target_P2:\n", P2_target)

# Initialize the 3-dimensional tensor P
P = np.zeros((3, 3, 3))  # Tensor for i2, i1, j

# Populate the tensor P using the provided equation
for i2 in range(3):
    for i1 in range(3):
        for j in range(3):
            P[i2, i1, j] = lambda_matrix_target[i1, i2] * P1_target[i1, j] + (1 - lambda_matrix_target[i1, i2]) * P2_target[i2, j]

print("Tensor P:\n", P)
vocab_size = 3

lambda_matrix_target:
 [[1.  0.  0.3]
 [0.  5.  4. ]
 [0.1 3.  0. ]]
target_P1:
 [[0.7 0.  0.3]
 [0.1 0.5 0.4]
 [1.  0.  0. ]]
target_P2:
 [[0.1 0.6 0.3]
 [0.  1.  0. ]
 [0.1 0.5 0.4]]
Tensor P:
 [[[ 0.69999999  0.          0.30000001]
  [ 0.1         0.60000002  0.30000001]
  [ 0.19        0.54000002  0.27000001]]

 [[ 0.          1.          0.        ]
  [ 0.5        -1.5         2.        ]
  [ 3.         -2.          0.        ]]

 [[ 0.28000001  0.34999999  0.37      ]
  [ 0.1         0.5         0.40000001]
  [ 0.1         0.5         0.40000001]]]


In [ ]:
# Final version
# Create test dataset from an empirical distribution
import numpy as np
import math

# Assuming P1, P2, target_lambda_matrix are already defined and appropriately normalized

# Set seed for reproducibility
np.random.seed(42)


def CreateDataset(dataset_size, lambda_matrix, P1, P2):

  C = np.zeros((3, 3, 3))
  for k in range(dataset_size):
      # Randomly sample i1 and i2
      i1, i2 = np.random.choice([0, 1, 2], 2)

      # Decide to choose from P1 or P2 based on lambda
      coin_flip = np.random.rand() < lambda_matrix[i1, i2]

      # Sample j based on the outcome of the coin flip
      if coin_flip:  # If true, choose from P1 using row i1
          chosen_row = P1[i1]
      else:  # Else, choose from P2 using row i2
          chosen_row = P2[i2]

      # Sample j from the chosen matrix row
      j = np.random.choice([0, 1, 2], p=chosen_row)

      # increment a count or sum probabilities in C[i2, i1, j]
      C[i2, i1, j] += 1

  return C


# Generate train data
train_size = 1000  # Number of samples in the test dataset
C_train = CreateDataset(train_size, lambda_matrix_target, P1_target, P2_target)

print('C_train =\n', C_train)

# Generate test data
test_size = 100  # Number of samples in the test dataset
C_test = CreateDataset(test_size, lambda_matrix_target, P1_target, P2_target)

# print('C_test =\n', C_test)




C_train =
 [[[ 95.   0.  37.]
  [ 14.  61.  34.]
  [ 23.  68.  31.]]

 [[  0.  93.   0.]
  [ 13.  65.  48.]
  [108.   0.   0.]]

 [[ 31.  42.  33.]
  [  7.  45.  42.]
  [ 17.  50.  43.]]]


In [ ]:
from scipy.sparse import csr_matrix, find
import math
import gc

alpha = 0.8
learning_rate = .0001  # Learning rate for updates
max_iterations = 10
rep_lambda = 5


def SmartInitialization_p1(C):
  ########## smart initialization for P1
  P1 = np.sum(C, axis=0)
  # Ensure P1 and P2 rows sum to 1 for probability matrices
  P1 /= P1.sum(axis=1, keepdims=True)
  # print("initial_P1:\n", P1)
  return P1


def SmartInitialization_p2(C):
  ########## smart initialization for P2
  P2 = np.sum(C, axis=1)
  # Ensure P1 and P2 rows sum to 1 for probability matrices
  P2 /= P2.sum(axis=1, keepdims=True)
  # print("initial_P2:\n", P2)
  return P2


P1 = SmartInitialization_p1(C_train)
P2 = SmartInitialization_p2(C_train)

P1 = csr_matrix(P1)
P2 = csr_matrix(P2)
print('P1:\n',P1.todense())
print('P2:\n',P2.todense())

# Set seed for reproducibility
random_seed = 42
random.seed(random_seed)  # Python's built-in random module
np.random.seed(random_seed)  # NumPy
lambda_matrix = np.random.rand(3, 3)

lambda_matrix = csr_matrix(lambda_matrix)
print('lambda_matrix:\n',lambda_matrix.todense())


###### initial
CT = np.transpose(C_train, axes=(1,0,2))
# print('CT:\n', CT)
CT = CT.reshape(3,9)
# print('CT_reshape:\n', CT)
CT = csr_matrix(CT)
# CT_nonzero = find(CT)
# print('CT_nonzero:\n', CT_nonzero)

def get_empirical_cross_entropy_sparse(C, P1_csr, P2_csr, lambda_csr, N):
    empirical_cross_entropy = 0
    count = 0
    C_nonzero = find(C)
    # Assuming C and P_hat can be aligned or P_hat's necessary values can be efficiently computed:
    for i, idx, c_val in zip(*C_nonzero):
        i2 = i
        i1, j = get_original_indices(idx, vocab_size)
        # Extract corresponding non-zero values from P1, P2, and lambda_matrix
        # Directly access values from CSR matrices
        p1_val = P1_csr[i1, j] if P1_csr[i1, j] != 0 else 0
        p2_val = P2_csr[i2, j] if P2_csr[i2, j] != 0 else 0
        lambda_val = lambda_csr[i1, i2] if lambda_csr[i1, i2] != 0 else 0
        # Compute the denominator for the update formula
        p_val = lambda_val * p1_val + (1 - lambda_val) * p2_val
        count += c_val

        if p_val > 0:
            empirical_cross_entropy -= c_val * math.log2(p_val)
        else:
          empirical_cross_entropy -= 0  # Or some default value, or you can raise an error
    return math.pow(2, empirical_cross_entropy / N), count


for iteration in range(max_iterations):

    # print('iteration:', iteration)

    # Calculate the denominator: sum of C across the last axis (j)
    # CT = C_train_csr_slice2
    CT = CT
    # print('CT:\n', CT.todense())

    # # Prevent division by zero by setting zero denominators to 1 (or add a small epsilon)
    # denominator_lambda[denominator_lambda == 0] = 1e-10

    lambda_updated = update_lambda_sparse(CT, P1, P2, lambda_matrix, vocab_size)
    # Compute the scaled difference
    scaled_diff = (lambda_updated - lambda_matrix).multiply(learning_rate)
    # Compute the new lambda_matrix
    lambda_matrix = lambda_matrix + scaled_diff
    # # Optional: Convert back to CSR if you had converted it before
    # lambda_matrix = csr_matrix(lambda_matrix_new)


    # lambda_expanded = lambda_matrix[:, :, np.newaxis]

    multipliers = [1]  # You can extend this list to include more multipliers
    # multipliers = list(range(1,10))
    if iteration == 0 or iteration > rep_lambda:    #only update lambda and P1 and P2 in first iteration and after that iterates only lambda and then after rep_lambda iterates P1 and P2 as well

      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1 or iteration == 15*rep_lambda+1:    #not work
      # if iteration == rep_lambda+1 or iteration == 5*rep_lambda+1 or iteration == 10*rep_lambda+1:
      # if iteration > rep_lambda and (iteration - 1) % rep_lambda == 0:   #the condition will be true for iterations like rep_lambda + 1, 2*rep_lambda + 1, 3*rep_lambda + 1, and so on.

      if iteration == 0:
        ## reseting P1 and P2 after rep_lamda so it measns that with smart initialization of P1 and P2 first update lambda with rep_lambda iterations and then reset P1 and P2 since they are good
        # P1 = SmartInitialization_p_sparse(C_train_csr, vocab_size)
        # P2 = SmartInitialization_p_sparse(C_train_csr_slice2, vocab_size)
        P1 = SmartInitialization_p1(C_train)
        P2 = SmartInitialization_p2(C_train)

        P1 = csr_matrix(P1)
        P2 = csr_matrix(P2)
        print('iteration initial',iteration)

      if any(iteration == m * rep_lambda + 1 for m in multipliers):
        P1 = SmartInitialization_p1(C_train)

        P2 = SmartInitialization_p2(C_train)
        print('iteration reset',iteration)

      elif iteration != 0:

        print('iteration update',iteration)

        # Update P1
        numerator_P1 = update_p1_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        A_slice1, n_i = Smoothing_P_sparse(numerator_P1, alpha=0.8)
        d_i_sparse1 = DistinctTokenSparse(numerator_P1)
        total_missing_mass1 = alpha * d_i_sparse1/n_i

        # Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
        # We're creating a dense array of the same shape as A_slice1 with each row having the same value
        missing_mass_per_vocab = np.repeat(total_missing_mass1 / vocab_size, vocab_size, axis=1)
        # Convert the missing_mass_per_vocab to a sparse matrix
        missing_mass_per_vocab_sparse = csr_matrix(missing_mass_per_vocab)

        # Add the missing_mass_per_vocab_sparse to A_slice1 element-wise (smoothing method to add absolute discount to each element and remove zero element)
        P1 = A_slice1 + missing_mass_per_vocab_sparse
        # print('P11111:\n',P1.todense())


        # # Update P2
        numerator_P2 = update_p2_sparse(CT, P1, P2, lambda_matrix, vocab_size)
        A_slice2, n_i = Smoothing_P_sparse(numerator_P2, alpha=0.8)
        d_i_sparse2 = DistinctTokenSparse(numerator_P2)
        total_missing_mass2 = alpha * d_i_sparse2/n_i
        # Convert missing_mass_per_vocab_sparse to a format that can be added to A_slice1
        # We're creating a dense array of the same shape as A_slice1 with each row having the same value
        missing_mass_per_vocab2 = np.repeat(total_missing_mass2 / vocab_size, vocab_size, axis=1)
        # Convert the missing_mass_per_vocab to a sparse matrix
        missing_mass_per_vocab_sparse2 = csr_matrix(missing_mass_per_vocab2)
        P2 = A_slice2 + missing_mass_per_vocab_sparse2
        # print('P22222:\n',P2.todense())
        # P2 = numerator_P2 / np.sum(numerator_P2, axis=1, keepdims=True)
        # P2 /= P2.sum(axis=1, keepdims=True)
    # if iteration % 2 == 0:
    #   P_hat = find_P_hat(lambda_matrix, P1, P2)
    #   # print('P_hat:', P_hat)
    #   N = test_size
    #   empirical_cross_entropy = get_empirical_cross_entropy_sparse(C_test, P_hat, N)
    #   print('empirical_cross_entropy:', empirical_cross_entropy)
        # Clear memory explicitly
        gc.collect()


# print('lambda_matrix_updated:\n',lambda_matrix)

# print('P1_updated:\n', P1)
# print('P1_updated_dense:\n', P1.todense())

# print('P2_updated:\n', P2)
# print('P2_updated_dense:\n', P2.todense())


# P1 = SmartInitialization_p1(C_train)
# P2 = SmartInitialization_p2(C_train)
# lambda_matrix, P1, P2 = MultiplicativeUpdate(C_train, lambda_matrix, P1, P2, max_iterations, rep_lambda, learning_rate)

# Print the updated lambda_matrix
print(f"Iteration {max_iterations}, Max change: {np.max(np.abs(lambda_matrix - lambda_matrix_target))}")
print("Updated lambda_matrix:\n", lambda_matrix.todense())
print(f"Iteration {max_iterations}, Max change: {np.max(np.abs(P1 - P1_target))}")
# Print out the final lambda matrix
print("Updated P1:\n", P1.todense())
print(f"Iteration {max_iterations}, Max change: {np.max(np.abs(P2 - P2_target))}")
print("Updated P2:\n", P2.todense())


# P_hat is a learned dataset
# Initialize the 3-dimensional tensor P_hat
P_hat = np.zeros((3, 3, 3))  # Tensor for i2, i1, j

# Populate the tensor P using the provided equation
for i2 in range(3):
    for i1 in range(3):
        for j in range(3):
            P_hat[i2, i1, j] = lambda_matrix[i1, i2] * P1[i1, j] + (1 - lambda_matrix[i1, i2]) * P2[i2, j]

print("Tensor P_hat:\n", P_hat)
print('-----------------------------------------')


N = test_size
C_test = C_test.reshape(3,9)
C_test = csr_matrix(C_test)
empirical_cross_entropy, count = get_empirical_cross_entropy_sparse(C_test, P1, P2, lambda_matrix, N)
print('empirical_cross_entropy:', empirical_cross_entropy)
# print('count:', count)


# empirical_cross_entropy: 276.6979117176241
# emp2 892.2182737491097
# count: 67598
# N: 81673
# test_size: 81673

# lambda_matrix_target:
#  [[1.  0.  0.3]
#  [0.  5.  4. ]
#  [0.1 3.  0. ]]
# target_P1:
#  [[0.7 0.  0.3]
#  [0.1 0.5 0.4]
#  [1.  0.  0. ]]
# target_P2:
#  [[0.1 0.6 0.3]
#  [0.  1.  0. ]
#  [0.1 0.5 0.4]]
# Tensor P:
#  [[[ 0.69999999  0.          0.30000001]
#   [ 0.1         0.60000002  0.30000001]
#   [ 0.19        0.54000002  0.27000001]]

#  [[ 0.          1.          0.        ]
#   [ 0.5        -1.5         2.        ]
#   [ 3.         -2.          0.        ]]

#  [[ 0.28000001  0.34999999  0.37      ]
#   [ 0.1         0.5         0.40000001]
#   [ 0.1         0.5         0.40000001]]]

P1:
 [[0.38066465 0.40785498 0.21148036]
 [0.10334347 0.51975684 0.3768997 ]
 [0.43529412 0.34705882 0.21764706]]
P2:
 [[0.36363636 0.3553719  0.28099174]
 [0.37003058 0.48318043 0.14678899]
 [0.17741935 0.44193548 0.38064516]]
lambda_matrix:
 [[0.37454012 0.95071431 0.73199394]
 [0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501]]
iteration initial 0
iteration reset 6
iteration update 7
iteration update 8
iteration update 9
Iteration 10, Max change: 4.843934166494029
Updated lambda_matrix:
 [[0.37451426 0.95070839 0.73198098]
 [0.59868978 0.15606583 0.15599935]
 [0.05807888 0.8662101  0.60103416]]
Iteration 10, Max change: 0.6092795785695052
Updated P1:
 [[0.28813304 0.60927958 0.10258738]
 [0.02696165 0.62771785 0.3453205 ]
 [0.81882175 0.11793447 0.06324378]]
Iteration 10, Max change: 0.49596619704929923
Updated P2:
 [[0.38906087 0.32402204 0.28691709]
 [0.13720668 0.5040338  0.35875952]
 [0.05136919 0.43528249 0.51334832]]
Tensor P_hat:
 [[[0.35126196 0.43085506 

In [ ]:
P1:
 [[0.38066465 0.40785498 0.21148036]
 [0.10334347 0.51975684 0.3768997 ]
 [0.43529412 0.34705882 0.21764706]]
P2:
 [[0.36363636 0.3553719  0.28099174]
 [0.37003058 0.48318043 0.14678899]
 [0.17741935 0.44193548 0.38064516]]
lambda_matrix:
 [[0.37454012 0.95071431 0.73199394]
 [0.59865848 0.15601864 0.15599452]
 [0.05808361 0.86617615 0.60111501]]
iteration: 0
iteration: 1
Iteration 2, Max change: 4.843866880999313
Updated lambda_matrix:
 [[0.37451983 0.95069715 0.73197227]
 [0.59872107 0.15613312 0.15600621]
 [0.05807985 0.86621161 0.60101067]]
Iteration 2, Max change: 0.5647058823529412
Updated P1:
 [[0.38066465 0.40785498 0.21148036]
 [0.10334347 0.51975684 0.3768997 ]
 [0.43529412 0.34705882 0.21764706]]
Iteration 2, Max change: 0.5168195718654434
Updated P2:
 [[0.36363636 0.3553719  0.28099174]
 [0.37003058 0.48318043 0.14678899]
 [0.17741935 0.44193548 0.38064516]]
Tensor P_hat:
 [[[0.3700138  0.37502786 0.25495835]
  [0.20779352 0.45379263 0.33841385]
  [0.36779824 0.35488908 0.27731269]]

 [[0.38014036 0.41156874 0.20829089]
  [0.32839189 0.48889122 0.18271689]
  [0.42656261 0.36527031 0.20816707]]

 [[0.32618928 0.4169895  0.25682122]
  [0.16586306 0.4540761  0.38006085]
  [0.33240484 0.3849136  0.28268156]]]
-----------------------------------------
empirical_cross_entropy: 2.833268352935607



Iteration 10, Max change: 4.843939722396137
Updated lambda_matrix:
 [[0.37450488 0.95071007 0.73197872]
 [0.5986896  0.15606028 0.15599884]
 [0.05807699 0.86621918 0.6010134 ]]
Iteration 10, Max change: 0.6156376224993734
Updated P1:
 [[0.29299369 0.61563762 0.09136869]
 [0.02315317 0.64957273 0.3272741 ]
 [0.84303833 0.10846669 0.04849498]]
Iteration 10, Max change: 0.5002168101794461
Updated P2:
 [[0.38695262 0.32137464 0.29167275]
 [0.13547244 0.49978319 0.36474437]
 [0.04817241 0.42995401 0.52187359]]
Tensor P_hat:
 [[[0.35176454 0.43157756 0.2166579 ]
  [0.16914967 0.51786342 0.31298691]
  [0.4134407  0.30900959 0.27754971]]

 [[0.28522947 0.60992717 0.10484336]
  [0.11794386 0.52315939 0.35889675]
  [0.74837959 0.16081733 0.09080308]]

 [[0.22737637 0.56587046 0.20675317]
  [0.04426943 0.46421427 0.49151629]
  [0.52589748 0.23673582 0.2373667 ]]]
-----------------------------------------
empirical_cross_entropy: 2.555986311895934